<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/Fine_tuningLLM_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Imports and Initial Setup

In the initial cells, essential libraries and dependencies are imported, setting up the environment for the fine-tuning process. We use a custom trainer, configure GPU settings, and set up necessary libraries like Hugging Face's `transformers` and `peft` (Parameter-Efficient Fine-Tuning).

In [1]:
!pip install --upgrade peft transformers bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType

In [3]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

**Load Datasets**

In [4]:
from datasets import load_dataset

# Load the Sanskrit dataset from OSCAR
dataset = load_dataset('oscar', 'unshuffled_deduplicated_sa', split='train')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/303k [00:00<?, ?B/s]

oscar.py:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/7121 [00:00<?, ? examples/s]

In [5]:
# Display a few samples

for i in range(3):
    print(f"Sample {i+1}:\n{dataset[i]['text']}\n")

Sample 1:
अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति । तस्‍य कानिचन् चित्राणि पूर्वमेव प्रकाशितानि सन्ति । द्वौ चलचित्रौ अपि प्रकाशितौ । तस्मिन् एव क्रमेण एतत् सीतास्‍वयंबर इति चलचित्रं प्रकाश्यते ।
लट् लकार: एकवचनम् द्विवचनम्बहुवचनम्प्रथमपुरुष:गच्‍छतिगच्‍छत:गच्‍छन्तिमध्‍यमपुरुष:गच्‍छसिगच्‍छथ:गच्‍छथउत्‍तमपुरुष:गच्‍छामिगच्‍छाव:गच्‍छाम:

Sample 2:
पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति; अन्याः संस्थित्यः अपि सन्ति । अधिकं ज्ञातुम् अत्र उपयोगस्य संस्थितिं पश्यतु ।

Sample 3:
स्थिते च कवचे देहे नास्ति मृत्युश्च जीविनाम् । अस्त्रे-शस्त्रे-जले-वह्नौ सिद्धिश्चेन्नास्ति संशयः ॥ ४॥
प्राच्यां मां पातु भूतेशः आग्नेय्यां पातु शङ्करः । दक्षिणे पातु मां रुद्रो नैॠत्यां स्थाणुरेव च ॥ ११॥



**Data Cleaning**

In [6]:
import re

def clean_text(example):
    text = example['text']
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove non-Sanskrit characters (retain Devanagari script)
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)
    return {'text': text}

# Apply the cleaning function
dataset = dataset.map(clean_text)

Map:   0%|          | 0/7121 [00:00<?, ? examples/s]

In [7]:
def filter_short_texts(example):
    return len(example['text'].split()) > 5  # Keep texts longer than 5 words

dataset = dataset.filter(filter_short_texts)

Filter:   0%|          | 0/7121 [00:00<?, ? examples/s]

In [ ]:
dataset[0]

In [ ]:
dataset[2]

**Load Model**

In [12]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/hf_token.txt', 'r') as f:
    hf_token = f.read().strip()

from huggingface_hub import login
login(token=hf_token)

model_name = 'google/gemma-2-2b-it'

Mounted at /content/drive


In [13]:
# Use a multilingual tokenizer or train a new one
# Load the tokenizer for Gemma 2 9B model

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

**Check whether Cuda is available**

In [14]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


**LORA Configurations**

In [15]:
from transformers import BitsAndBytesConfig

#bnb_config = BitsAndBytesConfig(load_in_8bit=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [17]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.CAUSAL_LM,
)

In [18]:
model = get_peft_model(model, lora_config)

In [19]:
for name, param in model.named_parameters():
    if 'lora' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

**Number of trainable parameters**

In [20]:
# Verify trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params} | All params: {all_params} | "
        f"Trainable percentage: {100 * trainable_params / all_params:.2f}%"
    )

print_trainable_parameters(model)

Trainable params: 3194880 | All params: 1605398784 | Trainable percentage: 0.20%


In [ ]:
#model.gradient_checkpointing_enable()

**Prepare Tokenized Dataset**

In [21]:
# Prepare dataset
def tokenize_function(examples):
    tokens = tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,
    )
    tokens['labels'] = tokens['input_ids'].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
print(tokenized_dataset[0])  # Check the first example in the tokenized dataset

Map:   0%|          | 0/6957 [00:00<?, ? examples/s]

{'id': 0, 'input_ids': [2, 236088, 235530, 42894, 235844, 54736, 235530, 235827, 36352, 2280, 40831, 236127, 117062, 196908, 33920, 17285, 124267, 54701, 22870, 18852, 46577, 235485, 9043, 8104, 45884, 235527, 21640, 10321, 124199, 11201, 56667, 236062, 18029, 213831, 235437, 153743, 161306, 60152, 235699, 172684, 117062, 56667, 3640, 86532, 235527, 21640, 52348, 237453, 118948, 96061, 5444, 237453, 27456, 235527, 172684, 13568, 237453, 21640, 10321, 22740, 212361, 18029, 12255, 235699, 82046, 60152, 236396, 12255, 235550, 35871, 65892, 9043, 117232, 235643, 104371, 14176, 22870, 118948, 96061, 5444, 235563, 12218, 22729, 87046, 18249, 21640, 8460, 235935, 235485, 8460, 31218, 15848, 235699, 236062, 235530, 26989, 52348, 52556, 236062, 235530, 120292, 235571, 235844, 235699, 236062, 235530, 26989, 54701, 236277, 235579, 121504, 225250, 235827, 73635, 22870, 235827, 73635, 235550, 235827, 73635, 86532, 99524, 171253, 121504, 225250, 235827, 73635, 130289], 'attention_mask': [1, 1, 1, 1,

**Customized Data Collator**

In [22]:
# Custom data collator
def custom_data_collator(features):
    batch = {}
    for key in features[0].keys():
        batch[key] = torch.stack([torch.tensor(f[key]) for f in features]).to('cuda')
    return batch

# Custom Trainer Initialization

A `CustomTrainer` instance is initialized with the model, training arguments, and dataset. This custom trainer coordinates the training process, using the arguments and model components configured earlier.

In [23]:
class CustomTrainer(Trainer):
    def prepare_inputs(self, inputs):
        # Ensure inputs are on GPU
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        return inputs

In [24]:
# Get a batch of data
batch = tokenized_dataset[:2]  # Take the first 2 samples

# Convert batch to tensors and move to GPU
inputs = {
    'input_ids': torch.tensor(batch['input_ids']).to('cuda'),
    'attention_mask': torch.tensor(batch['attention_mask']).to('cuda'),
    'labels': torch.tensor(batch['labels']).to('cuda'),
}

**Set model to evaluation mode**

In [25]:
model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    loss = outputs.loss
    print(f"Loss: {loss.item()}")
    print(f"Loss requires grad: {loss.requires_grad}")

Loss: 10.333832740783691
Loss requires grad: False


**Set model to training mode**

In [26]:
model.train()

outputs = model(**inputs)
loss = outputs.loss

print(f"Loss: {loss.item()}")
print(f"Loss requires grad: {loss.requires_grad}")

It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Loss: 10.333832740783691
Loss requires grad: True


In [27]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: requires_grad={param.requires_grad}, device={param.device}")

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.1.self_attn.q_pr

In [28]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Setting Training Arguments

Training arguments are configured here, including batch size, number of training epochs, learning rate, and other key settings. This configuration is essential for controlling the fine-tuning process.

In [29]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/results/',
    num_train_epochs=1,
    per_device_train_batch_size=10,  # Adjust based on GPU memory
    gradient_accumulation_steps=2,  # Adjust to maintain effective batch size
    learning_rate=1e-4,
    fp16=True,
    save_total_limit=2,
    save_steps=50,
    gradient_checkpointing=False,
    optim='adamw_bnb_8bit',
    dataloader_pin_memory=False
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [30]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=custom_data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


# Memory Management and Training Execution

To manage memory effectively on the GPU, we clear the CUDA cache before starting the actual training. Then, the `trainer.train()` command is used to start the fine-tuning.

The training progress, including the number of steps completed, current loss, and other details, is displayed to monitor the model's learning curve.

In [31]:
# Clear cache and start training
torch.cuda.empty_cache()

In [32]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=348, training_loss=2.8559182704180137, metrics={'train_runtime': 831.8461, 'train_samples_per_second': 8.363, 'train_steps_per_second': 0.418, 'total_flos': 1.0834020654317568e+16, 'train_loss': 2.8559182704180137, 'epoch': 1.0})

# Saving the Trained Model

Once training is complete, we save the fine-tuned model and tokenizer to a directory for future usage in generating Sanskrit text.


In [33]:
trainer.save_model('fine-tuned-gemma2-sanskrit-lora')
tokenizer.save_pretrained('fine-tuned-gemma2-sanskrit-lora')

('/kaggle/working/fine-tuned-gemma2-sanskrit-lora/tokenizer_config.json',
 '/kaggle/working/fine-tuned-gemma2-sanskrit-lora/special_tokens_map.json',
 '/kaggle/working/fine-tuned-gemma2-sanskrit-lora/tokenizer.model',
 '/kaggle/working/fine-tuned-gemma2-sanskrit-lora/added_tokens.json',
 '/kaggle/working/fine-tuned-gemma2-sanskrit-lora/tokenizer.json')

# Model Loading and Evaluation Setup

In this section, we load the fine-tuned model into evaluation mode for generating Sanskrit text. The `PeftModel` allows efficient loading and evaluation.

In [35]:
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map='auto',
    llm_int8_enable_fp32_cpu_offload=True  # Enables offloading to the CPU if needed
)

model = PeftModel.from_pretrained(model, 'fine-tuned-gemma2-sanskrit-lora')

# Set the model to evaluation mode
model.eval()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
          

# Text Generation Function

A function, `generate_text`, is defined for generating Sanskrit text based on a provided prompt. Parameters for temperature, top-p sampling, and maximum length control the creativity and diversity of the generated text.


In [36]:
def generate_text(prompt, max_length=128, num_return_sequences=1):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    # Generate output sequences
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,           # Use sampling for more diverse outputs
            temperature=0.7,          # Adjust temperature for creativity
            top_p=0.9,                # Use top-p sampling
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decode the generated tokens
    generated_texts = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts

# Example Prompt and Generated Text

Using the `generate_text` function, we provide an example Sanskrit prompt and generate text based on it to showcase the model's capability to create coherent Sanskrit sentences.

In [37]:
prompt = "पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति"
generated_texts = generate_text(prompt, max_length=128)
print("Generated Text:")
print(generated_texts[0])

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Generated Text:
पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति । साधनाः न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायोपदेशः । न्यायो


In [38]:
prompt = "अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति ।"
generated_texts = generate_text(prompt, max_length=128)

print("Generated Text:")
print(generated_texts[0])

Generated Text:
अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति ।  प्राङ्गशास्त्रस्य प्रथमं भागम् । प्राङ्गशास्त्रस्य द्वितीयं भागम् । प्राङ्गशास्त्रस्य तृतीयं भागम् । प्राङ्गशास्त्रस्य चतुर्थं भागम् ।  प्राङ्गशास्त्रस्य पञ्चमं भागम् । प्रख्यातं प्राङ्गशास्त्रम् ।  क्रीडिता रामलीला सम्‍प्रति


In [39]:

prompt = "सर्वे भवन्तु सुखिनः, सर्वे सन्तु निरामयाः।"
generated_texts = generate_text(prompt, max_length=128)

print("Generated Text:")
print(generated_texts[0])

Generated Text:
सर्वे भवन्तु सुखिनः, सर्वे सन्तु निरामयाः। सर्वेभ्यः पश्यन्तु मातानि शान्तिः। सन्तु कश्चिन् पश्यन्तु पितृभिः शान्तिः। पितृभिः शान्तिः। पितृभिः शान्तिः। पितृभिः शान्तिः। पितृभिः शान्तिः। पितृभिः शान्तिः। पितृभिः शान्तिः।
